<a href="https://colab.research.google.com/github/Richardesl10/Se-alesysistemasrichard/blob/main/Copia_final_de_PROYECTO_FINAL_GIGA_SCIENCE_EEGMI_GCPDS_3b7aed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
dggarciam94_giga_science_gcpds_path = kagglehub.dataset_download('dggarciam94/giga-science-gcpds')

print('Data source import complete.')


# Proyecto Final Señales y Sistemas 2024 -2

## **Objetivo**: Implementar técnicas de representación en tiempo y frecuencia para el reconocimiento de señales de electroencefalografía (EEG) en tareas de imaginación motora (Motor Imagery)

![sensorimotora](https://nepsa.es/wp-content/uploads/2021/10/Lengua-Mandibula-Labios-Cara-Ojo-Frente-Cuello-Pulgar-Dedos-Mano-Muneca-Codo-Brazo-Hombro-Maletero-Cadera-3.png)

![eegMI](https://figures.semanticscholar.org/288a54f091264377eccc99a19079c9387d66a78f/3-Figure2-1.png)

EEG signals can be noisy from a number of sources, including physiological artifacts and electromagnetic interference. They can also vary from person to person, which makes it harder to extract features and understand the signals. Additionally, this variability, influenced by genetic and cognitive factors, presents challenges for developing subject-independent solutions.

**Base de datos**: GiGaScience Database [https://gigadb.org/dataset/100295](https://gigadb.org/dataset/100295)

Ver Sección 3.1 en [Multimodal Explainability Using Class Activation Maps and Canonical Correlation for MI-EEG Deep Learning Classification](https://www.mdpi.com/2076-3417/14/23/11208)


## Instalamos las librerias necesarias

## Ejercicio 1
Consultar para qué sirven las siguientes librerías

Ejercicio 1

1. TensorFlow (tensorflow==2.15.0)

TensorFlow es una de las bibliotecas más populares para aprendizaje profundo (Deep Learning). Fue desarrollada por Google y se usa en múltiples aplicaciones, como visión por computadora, procesamiento de lenguaje natural y análisis de datos biomédicos, incluyendo EEG.

En el contexto del análisis de EEG, TensorFlow se usa para:

* Construir y entrenar redes neuronales convolucionales (CNN) y redes recurrentes (RNN) para la clasificación de señales EEG.
* Implementar modelos de aprendizaje supervisado y no supervisado para detectar patrones en los datos.
* Usar técnicas de interpretabilidad como Class Activation Maps (CAMs) para visualizar qué características de las señales EEG son más relevantes para la clasificación.

2. MNE (mne==1.6.0)

MNE-Python es una biblioteca especializada en el análisis de señales neurofisiológicas, especialmente EEG y MEG (magnetoencefalografía).

Sus principales funcionalidades incluyen:

* Carga de datos EEG desde formatos estándar (e.g., EDF, BrainVision, BDF).
* Preprocesamiento de señales EEG, como:
* Filtrado (pasabanda, notch).
* Eliminación de artefactos fisiológicos (como movimientos oculares).
* Re-referenciación y normalización.
* Análisis en el dominio del tiempo, frecuencia y espacio, permitiendo detectar actividad cerebral en diferentes bandas de frecuencia (theta, alpha, beta, gamma).
* Creación de mapas topográficos para visualizar la actividad cerebral en diferentes electrodos.
* Análisis de potenciales evocados (ERP) y conectividad cerebral.
* MNE es fundamental en el análisis de EEG, ya que proporciona herramientas para procesar señales en bruto antes de usarlas en modelos de  Machine Learning con TensorFlow o Braindecode.

3. Braindecode (braindecode==0.7)

Braindecode es una biblioteca basada en PyTorch diseñada para el análisis de EEG utilizando aprendizaje profundo. Fue desarrollada específicamente para clasificación de señales EEG en aplicaciones como Brain-Computer Interfaces (BCI).

Algunas características clave de Braindecode:

* Incluye modelos preentrenados para EEG, como EEGNet y ShallowConvNet.
* Permite entrenar redes neuronales profundas (CNNs y RNNs) en conjuntos de datos EEG.
* Es compatible con MNE, lo que permite integrar datos preprocesados con modelos de aprendizaje profundo.
* Se usa en tareas como clasificación de imágenes motoras (motor imagery) o detección de eventos en EEG clínico.
* Braindecode puede servir para experimentar con modelos de Deep Learning aplicados a EEG y probar diferentes arquitecturas para mejorar la clasificación de señales cerebrales.

4. GCPDS Databases
   
Esta es una librería desarrollada por el Grupo de Procesamiento de Señales y Datos (GCPDS) de la Universidad de Navarra. Su objetivo es facilitar el acceso a bases de datos de señales fisiológicas, incluyendo EEG.

Sus usos incluyen:

* Carga automática de datasets EEG de investigaciones previas.
* Integración con modelos de Machine Learning para pruebas y entrenamiento.
* Uso en experimentos de Brain-Computer Interfaces (BCI).
* Esta librería puede ser útil si deseas trabajar con bases de datos EEG previamente curadas y listas para el análisis sin necesidad de preprocesarlas manualmente.

In [ ]:
#!pip install tensorflow==2.15.0
!pip install mne==1.6.0
!pip install braindecode===0.7
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases

## Importamos algunas librerias necesarias

In [ ]:
from scipy.signal import resample
from scipy.signal import freqz, filtfilt, resample
from scipy.signal import butter as bw
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
#import tensorflow as tf
from gcpds.databases import GIGA_MI_ME
from sklearn.base import BaseEstimator, TransformerMixin

## Funciones necesarias para el preprocesamiento leve de los datos

In [ ]:
def load_GIGA(db,
              sbj,
              eeg_ch_names,
              new_fs,
              fs,
              f_bank=None,
              vwt=None,
              run=None):

    index_eeg_chs = db.format_channels_selectors(channels = eeg_ch_names) - 1

    #tf_repr = TimeFrequencyRpr(sfreq = fs, f_bank = f_bank, vwt = vwt)

    db.load_subject(sbj)
    if run == None:
        X, y = db.get_data(classes = ['left hand mi', 'right hand mi']) #Load MI classes, all channels {EEG}, reject bad trials, uV
    else:
        X, y = db.get_run(run, classes = ['left hand mi', 'right hand mi']) #Load MI classes, all channels {EEG}, reject bad trials, uV
    X = X[:, index_eeg_chs, :] #spatial rearrangement
    #X = np.squeeze(tf_repr.transform(X))
    #Resampling
    if new_fs == fs:
        pass#print('No resampling, since new sampling rate same.')
    else:
        print("Resampling from {:f} to {:f} Hz.".format(fs, new_fs))
        X = resample(X, int((X.shape[-1]/fs)*new_fs), axis = -1)

    return X, y

def butterworth_digital_filter(X, N, Wn, btype, fs, axis=-1, padtype=None, padlen=0, method='pad', irlen=None):
  """
  Apply digital butterworth filter
  INPUT
  ------
  1. X: (D array)
    array with signals.
  2. N: (int+)
    The order of the filter.
  3. Wn: (float+ or 1D array)
    The critical frequency or frequencies. For lowpass and highpass filters, Wn is a scalar; for bandpass and bandstop filters, Wn is a length-2 vector.
    For a Butterworth filter, this is the point at which the gain drops to 1/sqrt(2) that of the passband (the “-3 dB point”).
    If fs is not specified, Wn units are normalized from 0 to 1, where 1 is the Nyquist frequency (Wn is thus in half cycles / sample and defined as 2*critical frequencies / fs). If fs is specified, Wn is in the same units as fs.
  4. btype: (str) {‘lowpass’, ‘highpass’, ‘bandpass’, ‘bandstop’}
    The type of filter
  5. fs: (float+)
    The sampling frequency of the digital system.
  6. axis: (int), Default=1.
    The axis of x to which the filter is applied.
  7. padtype: (str) or None, {'odd', 'even', 'constant'}
    This determines the type of extension to use for the padded signal to which the filter is applied. If padtype is None, no padding is used. The default is ‘odd’.
  8. padlen: (int+) or None, Default=0
    The number of elements by which to extend x at both ends of axis before applying the filter. This value must be less than x.shape[axis] - 1. padlen=0 implies no padding.
  9. method: (str), {'pad', 'gust'}
    Determines the method for handling the edges of the signal, either “pad” or “gust”. When method is “pad”, the signal is padded; the type of padding is determined by padtype
    and padlen, and irlen is ignored. When method is “gust”, Gustafsson’s method is used, and padtype and padlen are ignored.
  10. irlen: (int) or None, Default=nONE
    When method is “gust”, irlen specifies the length of the impulse response of the filter. If irlen is None, no part of the impulse response is ignored.
    For a long signal, specifying irlen can significantly improve the performance of the filter.
  OUTPUT
  ------
  X_fil: (D array)
    array with filtered signals.
  """
  b, a = bw(N, Wn, btype, analog=False, output='ba', fs=fs)
  return filtfilt(b, a, X, axis=axis, padtype=padtype, padlen=padlen, method=method, irlen=irlen)

class TimeFrequencyRpr(BaseEstimator, TransformerMixin):
  """
  Time frequency representation of EEG signals.

  Parameters
  ----------
    1. sfreq:  (float) Sampling frequency in Hz.
    2. f_bank: (2D array) Filter banks Frequencies. Default=None
    3. vwt:    (2D array) Interest time windows. Default=None
  Methods
  -------
    1. fit(X, y=None)
    2. transform(X, y=None)
  """
  def __init__(self, sfreq, f_bank=None, vwt=None):
    self.sfreq = sfreq
    self.f_bank = f_bank
    self.vwt = vwt
# ------------------------------------------------------------------------------

  def _validation_param(self):
    """
    Validate Time-Frequency characterization parameters.
    INPUT
    -----
      1. self
    ------
      2. None
    """
    if self.sfreq <= 0:
      raise ValueError('Non negative sampling frequency is accepted')


    if self.f_bank is None:
      self.flag_f_bank = False
    elif self.f_bank.ndim != 2:
      raise ValueError('Band frequencies have to be a 2D array')
    else:
      self.flag_f_bank = True

    if self.vwt is None:
      self.flag_vwt = False
    elif self.vwt.ndim != 2:
      raise ValueError('Time windows have to be a 2D array')
    else:
      self.flag_vwt = True

# ------------------------------------------------------------------------------
  def _filter_bank(self, X):
    """
    Filter bank Characterization.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
    OUTPUT
    ------
      1. X_f: (4D array) set of filtered EEG signals, shape (trials, channels, time_samples, frequency_bands)
    """
    X_f = np.zeros((X.shape[0], X.shape[1], X.shape[2], self.f_bank.shape[0])) #epochs, Ch, Time, bands
    for f in np.arange(self.f_bank.shape[0]):
      X_f[:,:,:,f] = butterworth_digital_filter(X, N=5, Wn=self.f_bank[f], btype='bandpass', fs=self.sfreq)
    return X_f

# ------------------------------------------------------------------------------
  def _sliding_windows(self, X):
    """
    Sliding Windows Characterization.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
    OUTPUT
    ------
      1. X_w: (4D array) shape (trials, channels, window_time_samples, number_of_windows)
    """
    window_lenght = int(self.sfreq*self.vwt[0,1] - self.sfreq*self.vwt[0,0])
    X_w = np.zeros((X.shape[0], X.shape[1], window_lenght, self.vwt.shape[0]))
    for w in np.arange(self.vwt.shape[0]):
        X_w[:,:,:,w] = X[:,:,int(self.sfreq*self.vwt[w,0]):int(self.sfreq*self.vwt[w,1])]
    return X_w

# ------------------------------------------------------------------------------
  def fit(self, X, y=None):
    """
    fit.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
      2. y: (1D array) target labels. Default=None
    OUTPUT
    ------
      1. None
    """
    pass

# ------------------------------------------------------------------------------
  def transform(self, X, y=None):
    """
    Time frequency representation of EEG signals.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, times)
    OUTPUT
    ------
      1. X_wf: (5D array) Time-frequency representation of EEG signals, shape (trials, channels, window_time_samples, number_of_windows, frequency_bands)
    """
    self._validation_param()     #Validate sfreq, f_freq, vwt

    #Avoid edge effects of digital filter, 1st:fbk, 2th:vwt
    if self.flag_f_bank:
        X_f = self._filter_bank(X)
    else:
        X_f = X[:,:,:,np.newaxis]

    if self.flag_vwt:
      X_wf = []
      for f in range(X_f.shape[3]):
        X_wf.append(self._sliding_windows(X_f[:,:,:,f]))
      X_wf = np.stack(X_wf, axis=-1)
    else:
      X_wf = X_f[:,:,:,np.newaxis,:]

    return X_wf

#plot eeg
def plot_eeg(X,tv,ax,channels,esp=2,title=None):
    # X in CH x Samples
    n_canales = X.shape[0]

    for ch in range(n_canales): # canales
            xx = X[ch]
            xx = xx - np.mean(xx)
            xx = xx/np.max(abs(xx))
            ax.plot(tv, xx +(ch * esp), label=channels[ch])  # Desplazamos cada canal para visualización
    ax.set_yticks(range(0, esp * n_canales, esp), channels)  # Etiquetas en el eje Y
    ax.set_xlabel("Tiempo [s]")
    ax.set_ylabel("Canales EEG [$\mu$V]")
    ax.set_title(title)
    ax.grid(True)
    ax.set_xlim([min(tv)-0.01,max(tv)+0.01])
    ax.set_ylim([-esp,n_canales*esp+0.01])







## Establecemos el protocolo de pruebas y la configuración del montaje EEG

Describir el protocolo de captura de datos y el montaje utilizado


![mi](https://www.mdpi.com/diagnostics/diagnostics-13-01122/article_deploy/html/images/diagnostics-13-01122-g001.png)
![montaje](https://www.mdpi.com/applsci/applsci-14-11208/article_deploy/html/images/applsci-14-11208-g001.png)

###solucion


Protocolo de Captura de Datos

Preparación del sujeto:
El sujeto se sienta en una silla cómoda con electrodos colocados en su cuero cabelludo, siguiendo el sistema de colocación 10-20.

Se establecen las conexiones con un sistema de adquisición de EEG (electroencefalografía).

1.1  Presentación de estímulos:
Una pantalla frente al sujeto presenta señales visuales (visual cue) indicando la tarea mental a realizar.

Las instrucciones pueden indicar la imaginación del movimiento de la mano izquierda o derecha.

1.2. Registro del EEG:
Durante el periodo indicado (por ejemplo, 5 segundos), el sujeto realiza la tarea mental de imaginar el movimiento de la mano especificada.

Los electrodos capturan la actividad cerebral en áreas motoras relevantes.

 1.3. Intervalos entre pruebas:
Se incluyen periodos de descanso entre ensayos de aproximadamente 0.1-0.8 segundos.

Montaje Utilizado

2.1 Electrodos EEG:
Colocados en posiciones estratégicas sobre la corteza motora, con énfasis en la región central del cuero cabelludo (C3, Cz, C4).
Se destacan en rojo los electrodos más relevantes para la tarea.
Dispositivos:
Un PC para procesar los datos y presentar estímulos visuales.
Una interfaz para la adquisición y procesamiento de las señales EEG.


In [ ]:
channels = ['Fp1','Fpz','Fp2',
            'AF7','AF3','AFz','AF4','AF8',
            'F7','F5','F3','F1','Fz','F2','F4','F6','F8',
            'FT7','FC5','FC3','FC1','FCz','FC2','FC4','FC6','FT8',
            'T7','C5','C3','C1','Cz','C2','C4','C6','T8',
            'TP7','CP5','CP3','CP1','CPz','CP2','CP4','CP6','TP8',
            'P9','P7','P5','P3','P1','Pz','P2','P4','P6','P8','P10',
            'PO7','PO3','POz','PO4','PO8',
            'O1','Oz','O2',
            'Iz']

areas = {
    'Frontal': ['Fpz', 'AFz', 'Fz', 'FCz'],
    'Frontal Right': ['Fp2','AF4','AF8','F2','F4','F6','F8',],
    'Central Right': ['FC2','FC4','FC6','FT8','C2','C4','C6','T8','CP2','CP4','CP6','TP8',],
    'Posterior Right': ['P2','P4','P6','P8','P10','PO4','PO8','O2',],
    #'Central': ['Cz'],
    'Posterior': ['CPz','Pz', 'Cz','POz','Oz','Iz',],
    'Posterior Left': ['P1','P3','P5','P7','P9','PO3','PO7','O1',],
    'Central Left': ['FC1','FC3','FC5','FT7','C1','C3','C5','T7','CP1','CP3','CP5','TP7',],
    'Frontal Left': ['Fp1','AF3','AF7','F1','F3','F5','F7',],
}

arcs = [
    #'hemispheres',
    'areas',
    'channels',
]

## Definimos la ruta y los argumentos para la carga de los datos de EEG

In [ ]:
db = GIGA_MI_ME('/kaggle/input/giga-science-gcpds/GIGA_MI_ME')
#ti = 0
#tf = 7
new_fs = 256.
load_args = dict(db = db,
                 eeg_ch_names = channels,
                 fs = db.metadata['sampling_rate'],
                 #f_bank = np.asarray([[4., 40.]]),
                 #vwt = np.asarray([[ti, tf]]), #2.5 - 5 MI
                 new_fs = new_fs)

## Cargamos los datos según el sujeto que se quiera

Si se quiere cargar los datos de todos los sujetos, aplicar un ciclo que itere la lista de sujetos y de esta forma se cargara uno por uno dependiendo lo que se desee realizar.

Por ejemplo:

for i in sbj:
    X, y = load_GIGA(sbj=sbj, **load_args)

In [ ]:
db = GIGA_MI_ME('/kaggle/input/giga-science-gcpds/GIGA_MI_ME')
sbj = 35
new_fs = 256.
load_args = dict(db = db,
                 eeg_ch_names = channels,
                 fs = db.metadata['sampling_rate'],
                 #f_bank = np.asarray([[4., 40.]]),
                 #vwt = np.asarray([[ti, tf]]), #2.5 - 5 MI
                 new_fs = new_fs)
X, y = load_GIGA(sbj=sbj, **load_args)

In [ ]:
y

In [ ]:
print(f'X con {X.shape[0]} intentos; {X.shape[1]} canales; {X.shape[2]} muestras No. de segundos {X.shape[2]/new_fs}')

In [ ]:
X.shape

In [ ]:

# Lista de canales relevantes (FC, C y CP)
canales_de_interes = [i for i, ch in enumerate(channels) if ch.startswith(('FC', 'C', 'CP'))]

# Filtrar X para quedarnos solo con estos canales
X_filtrado = X[:, canales_de_interes, :]

print(f"Nuevo shape: {X_filtrado.shape}")  # (intentos, cantidad de canales seleccionados, muestras)

# Promediar sobre intentos
X_promedio = np.mean(X_filtrado, axis=0)  # (cantidad de canales seleccionados, muestras)

X_promedio.shape

In [ ]:
X_promedio.shape

In [ ]:
# Definir el rango de tiempo
trial = 0  # Intento a graficar (aunque ahora usamos la señal promediada)
ti = 0  # Tiempo inicial en segundos
tf = X_promedio.shape[1] / new_fs  # Tiempo final en segundos
tv = np.arange(ti, tf, 1 / new_fs)  # Vector de tiempo

# Crear figura
fig, ax = plt.subplots(1, 1, figsize=(8, 8), sharex=True)

# Graficar la señal promediada usando los canales de interés
plot_eeg(X_promedio, tv, ax=ax, channels=[channels[i] for i in canales_de_interes], title='EEG Promediado')

plt.show()


In [ ]:
sbj = 1
X, y = load_GIGA(sbj=sbj, **load_args)

# Lista de canales relevantes (FC, C y CP)
canales_de_interes = [i for i, ch in enumerate(channels) if ch.startswith(('FC', 'C', 'CP'))]

# Filtrar X para quedarnos solo con estos canales
X_filtrado = X[:, canales_de_interes, :]

print(f"Nuevo shape: {X_filtrado.shape}")  # (intentos, cantidad de canales seleccionados, muestras)

# Promediar sobre intentos
X_promedio = np.mean(X_filtrado, axis=0)  # (cantidad de canales seleccionados, muestras)

X_promedio.shape

# Definir el rango de tiempo
trial = 0  # Intento a graficar (aunque ahora usamos la señal promediada)
ti = 0  # Tiempo inicial en segundos
tf = X_promedio.shape[1] / new_fs  # Tiempo final en segundos
tv = np.arange(ti, tf, 1 / new_fs)  # Vector de tiempo

# Crear figura
fig, ax = plt.subplots(1, 1, figsize=(8, 8), sharex=True)

# Graficar la señal promediada usando los canales de interés
plot_eeg(X_promedio, tv, ax=ax, channels=[channels[i] for i in canales_de_interes], title='EEG Promediado')

plt.show()


In [ ]:
# Definir el rango de tiempo
trial = 0  # Intento a graficar (aunque ahora usamos la señal promediada)
ti = 0  # Tiempo inicial en segundos
tf = X_promedio.shape[1] / new_fs  # Tiempo final en segundos
tv = np.arange(ti, tf, 1 / new_fs)  # Vector de tiempo

# Crear figura
fig, ax = plt.subplots(1, 1, figsize=(8, 8), sharex=True)

# Graficar la señal promediada usando los canales de interés
plot_eeg(X_promedio, tv, ax=ax, channels=[channels[i] for i in canales_de_interes], title='EEG Promediado')

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Variable global para almacenar la información procesada de cada sujeto.
# Estructura: { sbj: { 'left': X_left_avg, 'right': X_right_avg, 'time': tv } }
global_eeg_promedios = {}

# Lista de índices de canales de interés: se filtran los canales que comienzan con "FC", "C" o "CP"
canales_de_interes = [i for i, ch in enumerate(channels) if ch.startswith(('FC', 'C', 'CP'))]

# Iterar sobre los 52 sujetos
for sbj in range(28, 53):

    if (29 == sbj or sbj == 34):
        pass
    else:
        print(f"Procesando sujeto {sbj}...")

        # Cargar datos del sujeto. load_GIGA retorna X (EEG) y y (etiquetas: 0 y 1)
        X, y = load_GIGA(sbj=sbj, **load_args)

        # Filtrar los canales de interés: X tiene forma (intentos, canales, muestras)
        X_filtrado = X[:, canales_de_interes, :]

        # Separar ensayos según la mano:
        # Se asume: 0 -> mano izquierda y 1 -> mano derecha.
        idx_left = (y == 0)
        idx_right = (y == 1)

        # Verificar que existan ensayos para cada condición
        if np.sum(idx_left) == 0:
            print(f"Advertencia: Sujeto {sbj} no tiene ensayos para mano izquierda.")
            continue
        if np.sum(idx_right) == 0:
            print(f"Advertencia: Sujeto {sbj} no tiene ensayos para mano derecha.")
            continue

        # Extraer y promediar los intentos para cada mano
        X_left = X_filtrado[idx_left, :, :]    # Ensayos de mano izquierda, forma: (n_trials_left, canales, muestras)
        X_right = X_filtrado[idx_right, :, :]   # Ensayos de mano derecha, forma: (n_trials_right, canales, muestras)

        X_left_avg = np.mean(X_left, axis=0)      # (canales, muestras)
        X_right_avg = np.mean(X_right, axis=0)    # (canales, muestras)

        # Definir el vector de tiempo basado en new_fs
        ti = 0
        tf_total = X_left_avg.shape[1] / new_fs  # Se asume que ambos tienen el mismo número de muestras
        tv = np.arange(ti, tf_total, 1 / new_fs)

        # Almacenar los promedios y el vector de tiempo en la variable global
        global_eeg_promedios[sbj] = {'left': X_left_avg, 'right': X_right_avg, 'time': tv}

        # Crear figura con dos subplots (uno para cada mano)
        fig, ax = plt.subplots(2, 1, figsize=(8, 10), sharex=True)

        # Graficar mano izquierda (usando únicamente los canales filtrados)
        plot_eeg(X_left_avg, tv, ax=ax[0], channels=[channels[i] for i in canales_de_interes],
             title=f'EEG Promediado Mano Izquierda - Sujeto {sbj}')

        # Graficar mano derecha
        plot_eeg(X_right_avg, tv, ax=ax[1], channels=[channels[i] for i in canales_de_interes],
             title=f'EEG Promediado Mano Derecha - Sujeto {sbj}')
        ax[1].set_xlabel("Tiempo [s]")

        # Mostrar la figura en pantalla
        plt.show()
        plt.close(fig)

        print(f"Mostrado y almacenado sujeto {sbj}")

    print("Procesamiento completado para todos los sujetos.")



## Visualización de las señales de EEG en el tiempo

In [ ]:
#graficar canales promedio
trial = 0
ti = 0 # ti
tf = 7 # tf
tv = np.arange(ti,tf,1/new_fs)

#Señal cruda
fig,ax = plt.subplots(1,1,figsize=(8,8),sharex = True)
# Graficar cada canal en un subplot banda respectiva

plot_eeg(X[trial],tv,ax=ax,channels=channels,title='EEG original')
plt.show()

# Ejercicio 2

Discuta la gráfica anterior

Ejercico 2


En la gráfica de EEG se observan múltiples canales representados por líneas de diferentes colores, cada una reflejando la actividad eléctrica de un electrodo colocado en el cuero cabelludo. Entre los segundos 2 y 5, se pueden notar variaciones en la amplitud de las señales en las regiones central (C3, Cz, C4), frontal central (FCz) y central posterior (CPz), lo que sugiere cambios en la activación neuronal. Se evidencia una disminución en la amplitud de ciertas frecuencias, especialmente en la banda Alpha (8-13 Hz), lo que es característico cuando una persona imagina movimientos. Además, se pueden notar pequeños picos y fluctuaciones en algunas señales, lo que podría estar asociado con la dinámica del procesamiento motor y la concentración del sujeto durante la tarea de imaginación. Conforme el tiempo avanza, la actividad tiende a estabilizarse, indicando que el cerebro regresa a un estado más neutral una vez finalizada la tarea mental.

En la gráfica de EEG, el intervalo entre 2 y 5 segundos es clave, ya que en este tiempo el sujeto está imaginando mover alguno de sus brazos. En la región central del cerebro (canales C3, Cz y C4), se pueden notar cambios en las ondas cerebrales que reflejan la activación de áreas relacionadas con el movimiento. Si el sujeto imagina mover el brazo derecho, se espera mayor actividad en el lado izquierdo del cerebro (C3), mientras que si imagina mover el brazo izquierdo, la actividad aumenta en el lado derecho (C4). Durante este proceso, se observa una disminución en ciertas ondas cerebrales, lo que indica que el cerebro está "preparándose" para ejecutar el movimiento, aunque este solo exista en su imaginación.

En las regiones frontal central y central posterior, también se perciben cambios en la actividad cerebral que muestran cómo el cerebro planifica y procesa la acción. En la zona frontal (FCz), el cerebro parece estar organizando y decidiendo el movimiento, mientras que en la parte posterior central (CPz), es posible que esté procesando la sensación del movimiento imaginado. Además, hacia el final de cada intervalo, las ondas cerebrales empiezan a estabilizarse nuevamente, lo que podría indicar que el cerebro "finaliza" la tarea imaginada y vuelve a su estado de reposo.

Nota: Recuerde el concepto de ritmos cerebrales

![montaje](https://cdn.shopify.com/s/files/1/0348/7053/files/storage.googleapis.com-486681944373284_61cb9936-f6c2-493d-8402-3426d7f5a049_1024x1024.jpg?v=1689309340)



In [ ]:
# filtramos trials completos en ritmos cerebrales utilizando filtros IIR


f_bank = np.array([[0.5,4.],[4., 8.],[8.,13.],[13.,32.],[32.,100.]])
vwt = np.asarray([[ti, tf]]) #2.5 - 5 MI 0 - 7 trial completo
tf_repr = TimeFrequencyRpr(sfreq = new_fs, f_bank = f_bank)

Xrc = np.squeeze(tf_repr.transform(X))

Xrc.shape

# Ejercicio 3

Expliqué cómo se calcularon cada una de las 5 dimensiones del arreglo Xrc

Ejercicio 3
Expliqué cómo se calcularon cada una de las 5 dimensiones del arreglo Xrc

El canal Xrc tiene 5 dimensiones aunque en el codigo solo se muestran 4 de manera explicita (n_trials, n_canales, n_muestras, n_bands). Sin embargo, hay una quinta dimensión que no se menciona directamente (n_sujetos).

n_trials : Es el numero de la prueba que estamos ejecutando, cada sujeto tiene 200 pruebas realizadas

n_canales: Corresponde al número de electrodos colocados en la cabeza del sujeto para captar la actividad EEG.

n_muestras: Se determina por la duración del ensayo y la frecuencia de muestreo. Por ejemplo se muestrea a 256 Hz y cada ensayo dura 3 segundos, el número de muestras es: 256 * 3 = 768

n_bands: Se obtiene aplicando filtros pasa-banda sobre la señal original, se divide en las bandas estandar: Delta, Theta, Alpha, Beta, Gamma

n_sujetos: Se separan los datos de cada persona en el experimento.

In [ ]:
import matplotlib.pyplot as plt

ritmo = ['delta','theta','alpha','beta','gamma']
trial = 25
n_trials, n_canales, n_muestras, n_bands = Xrc.shape  # Simulación de datos

esp = 2 #espaciado canales
fig,ax = plt.subplots(5,1,figsize=(8,40))
# Graficar cada canal en un subplot banda respectiva
for b in range(f_bank.shape[0]): #bandas
    plot_eeg(Xrc[trial,:,:,b],tv,ax=ax[b],channels=channels,title=f'EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]}')
plt.show()

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

# Variable global para almacenar la información procesada de cada sujeto.
# Estructura:
# global_eeg_promedios[sbj] = { 'left': left_filtered, 'right': right_filtered, 'time': tv }
# Donde left/right tienen forma (canales, muestras, bandas)
global_eeg_promedios1 = {}

# Lista de índices de canales de interés: canales que comienzan con "FC", "C" o "CP"
canales_de_interes = [i for i, ch in enumerate(channels) if ch.startswith(('FC', 'C', 'CP'))]

# Instanciar el transformador de tiempo-frecuencia usando el banco de filtros (f_bank) y la nueva frecuencia de muestreo
tf_repr = TimeFrequencyRpr(sfreq=new_fs, f_bank=f_bank, vwt=None)

# Iterar sobre los 52 sujetos
for sbj in range(1, 10):
    if(29 == sbj or 34 == sbj):
        pass
    else:
        print(f"Procesando sujeto {sbj}...")

        # Cargar datos: load_GIGA retorna X (EEG) y y (etiquetas: 0 para mano izquierda, 1 para mano derecha)
        X, y = load_GIGA(sbj=sbj, **load_args)

        # Filtrar para quedarse solo con los canales de interés
        # X tiene forma (trials, canales, muestras)
        X_filtrado = X[:, canales_de_interes, :]

        # Separar ensayos según la mano
        idx_left = (y == 0)
        idx_right = (y == 1)

        if np.sum(idx_left) == 0:
            print(f"Advertencia: Sujeto {sbj} no tiene ensayos para mano izquierda.")
            continue
        if np.sum(idx_right) == 0:
            print(f"Advertencia: Sujeto {sbj} no tiene ensayos para mano derecha.")
            continue

        # Extraer ensayos por condición
        X_left = X_filtrado[idx_left, :, :]   # forma: (n_trials_left, canales, muestras)
        X_right = X_filtrado[idx_right, :, :] # forma: (n_trials_right, canales, muestras)

        # Aplicar filtrado en bandas para cada condición:
        # La función transform de tf_repr devuelve una salida de forma (trials, canales, muestras, 1, bandas)
        Xrc_left = tf_repr.transform(X_left)
        Xrc_left = np.squeeze(Xrc_left, axis=3)   # Ahora: (trials, canales, muestras, bandas)
        Xrc_right = tf_repr.transform(X_right)
        Xrc_right = np.squeeze(Xrc_right, axis=3)  # (trials, canales, muestras, bandas)

        # Promediar sobre los ensayos (trials) para cada condición
        left_filtered = np.mean(Xrc_left, axis=0)   # (canales, muestras, bandas)
        right_filtered = np.mean(Xrc_right, axis=0)   # (canales, muestras, bandas)

        # Definir el vector de tiempo (suponiendo new_fs es la nueva frecuencia de muestreo)
        ti = 0
        tf_total = left_filtered.shape[1] / new_fs  # Asumimos misma cantidad de muestras en ambas condiciones
        tv = np.arange(ti, tf_total, 1 / new_fs)

        # Almacenar la información en la variable global para uso posterior
        global_eeg_promedios1[sbj] = {'left': left_filtered, 'right': right_filtered, 'time': tv}

        # Crear figura con 2 filas (una por condición) y N columnas (N = número de bandas, p.ej., 5)
        n_bands = f_bank.shape[0]
        fig, axes = plt.subplots(2, n_bands, figsize=(4*n_bands, 8), sharey=True, sharex=True)

        # Si solo hay una columna, convertir axes a 2D para evitar errores
        if n_bands == 1:
            axes = np.expand_dims(axes, axis=1)

        # Graficar para cada banda
        for b in range(n_bands):
            title_left = f'Izq: {f_bank[b,0]}-{f_bank[b,1]} Hz'
            title_right = f'Der: {f_bank[b,0]}-{f_bank[b,1]} Hz'
            # plot_eeg espera datos con forma (canales, muestras)
            plot_eeg(left_filtered[:, :, b], tv, ax=axes[0, b],
                     channels=[channels[i] for i in canales_de_interes],
                     title=title_left)
            plot_eeg(right_filtered[:, :, b], tv, ax=axes[1, b],
                     channels=[channels[i] for i in canales_de_interes],
                     title=title_right)

        # Ajustar ventana temporal para visualizar usando slider
        window_length = 7  # duración en segundos de la ventana visible inicialmente
        initial_start = 0
        initial_end = initial_start + window_length
        for ax in axes.flatten():
            ax.set_xlim(initial_start, initial_end)

        plt.suptitle(f"EEG Filtrado por Banda - Sujeto {sbj}\n(Arriba: Mano Izquierda, Abajo: Mano Derecha)")
        plt.show()
        plt.close(fig)

        print(f"Procesado y mostrado sujeto {sbj}")

    print("Procesamiento completado para todos los sujetos.")


In [22]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

# Variable global para almacenar la información procesada de cada sujeto.
# Estructura:
# global_eeg_promedios[sbj] = { 'left': left_filtered, 'right': right_filtered, 'time': tv }
# Donde left/right tienen forma (canales, muestras, bandas)
global_eeg_promedios = {}

# Lista de índices de canales de interés
canales_de_interes = [i for i, ch in enumerate(channels) if ch.startswith(('FC', 'C', 'CP'))]

# Instanciar el transformador de tiempo-frecuencia
tf_repr = TimeFrequencyRpr(sfreq=new_fs, f_bank=f_bank, vwt=None)

# Iterar sobre un rango de sujetos
for sbj in range(1, 52):
    if sbj in [29, 34]:
        pass
    else:
        print(f"Procesando sujeto {sbj}...")

        # Cargar datos originales: X (trials, n_canales, n_muestras) y y (etiquetas)
        X, y = load_GIGA(sbj=sbj, **load_args)

        # En lugar de descartar los canales, creamos una copia con ceros
        # del mismo tamaño que X
        X_zero = np.zeros_like(X)  # Mantiene la forma (trials, n_canales, n_muestras)

        # Rellenar con los datos originales solo en canales_de_interes
        X_zero[:, canales_de_interes, :] = X[:, canales_de_interes, :]

        # Separar ensayos según la mano
        idx_left = (y == 0)
        idx_right = (y == 1)

        if np.sum(idx_left) == 0:
            print(f"Advertencia: Sujeto {sbj} no tiene ensayos para mano izquierda.")
            continue
        if np.sum(idx_right) == 0:
            print(f"Advertencia: Sujeto {sbj} no tiene ensayos para mano derecha.")
            continue

        # Extraer ensayos por condición usando X_zero (con ceros fuera de canales_de_interes)
        X_left = X_zero[idx_left, :, :]    # (n_trials_left, n_canales, n_muestras)
        X_right = X_zero[idx_right, :, :]  # (n_trials_right, n_canales, n_muestras)

        # Aplicar filtrado en bandas
        # tf_repr.transform devuelve (trials, canales, muestras, 1, bandas)
        Xrc_left = tf_repr.transform(X_left)
        Xrc_left = np.squeeze(Xrc_left, axis=3)   # => (trials, n_canales, n_muestras, n_bands)
        Xrc_right = tf_repr.transform(X_right)
        Xrc_right = np.squeeze(Xrc_right, axis=3) # => (trials, n_canales, n_muestras, n_bands)

        # Promediar sobre los ensayos => (n_canales, n_muestras, n_bands)
        left_filtered = np.mean(Xrc_left, axis=0)
        right_filtered = np.mean(Xrc_right, axis=0)

        # Definir el vector de tiempo
        tf_total = left_filtered.shape[1] / new_fs
        tv = np.arange(0, tf_total, 1 / new_fs)

        # Guardar en la variable global
        global_eeg_promedios[sbj] = {
            'left':  left_filtered,   # (n_canales, n_muestras, n_bands)
            'right': right_filtered,  # (n_canales, n_muestras, n_bands)
            'time':  tv               # (n_muestras,)
        }


        print(f"Procesado y mostrado sujeto {sbj}")

print("Procesamiento completado para todos los sujetos.")


Procesando sujeto 1...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 1
Procesando sujeto 2...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 2
Procesando sujeto 3...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 3
Procesando sujeto 4...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 4
Procesando sujeto 5...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 5
Procesando sujeto 6...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 6
Procesando sujeto 7...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 7
Procesando sujeto 8...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 8
Procesando sujeto 9...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 9
Procesando sujeto 10...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 10
Procesando sujeto 

Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 30
Procesando sujeto 31...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 31
Procesando sujeto 32...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 32
Procesando sujeto 33...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 33
Procesando sujeto 35...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 35
Procesando sujeto 36...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 36
Procesando sujeto 37...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 37
Procesando sujeto 38...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 38
Procesando sujeto 39...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 39
Procesando sujeto 40...
Resampling from 512.000000 to 256.000000 Hz.
Procesado y mostrado sujeto 40
Procesando sujeto 41...


## Visualización de las señales de EEG en la frecuencia

In [ ]:
#señal orignal
Xwo = np.fft.rfft(X,axis=-1)
vfreq = np.fft.rfftfreq(X.shape[2],1/new_fs)

Xwo.shape
plt.plot(vfreq,20*np.log10(np.abs(Xwo[trial])).T)
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Magnitud [dB]')
plt.title('Eespectro Señal EEG original')
plt.show()


## Ejercicio 4

Discuta la gráfica anterior

punto 4
La gráfica representa el espectro de frecuencia de una señal EEG original, mostrando la distribución de su energía en diferentes frecuencias. En el eje X se observa el rango de frecuencias desde 0 hasta aproximadamente 125 Hz, mientras que el eje Y representa la magnitud en decibeles, indicando la intensidad de la señal en cada frecuencia. Destaca un pico pronunciado en las frecuencias más bajas, alrededor de los 0-5 Hz, lo que sugiere la presencia de ondas delta o posibles artefactos fisiológicos, como movimientos oculares o actividad de fondo. A medida que la frecuencia aumenta, la magnitud de la señal decrece rápidamente, lo que es característico de las señales EEG, ya que la mayor parte de la actividad cerebral se concentra en frecuencias bajas.

A partir de los 30 Hz, la señal muestra un comportamiento más irregular y disperso, lo que podría indicar la presencia de ruido electromagnético o la actividad de bandas como beta y gamma, asociadas con procesos cognitivos y motores. La distribución observada refuerza la idea de que las señales EEG tienen un espectro dominado por bajas frecuencias, con una actividad progresivamente más débil a medida que la frecuencia aumenta.

In [ ]:
#espectro señales filtradas
Xwb = np.fft.rfft(Xrc,axis=2)

Xwb.shape

In [ ]:
#espectro señales filtradas por bandas - ritmos cerebrales

fig,ax = plt.subplots(5,1,figsize=(8,40))
# Graficar cada canal en un subplot banda respectiva
for b in range(f_bank.shape[0]): #bandas
    ax[b].plot(vfreq,20*np.log10(np.abs(Xwb[trial,:,:,b])).T)
    ax[b].set_xlabel('Frecuencia [Hz]')
    ax[b].set_ylabel('Magnitud [dB]')
    ax[b].set_title(f'Esepctro EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]}')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Iterar sobre cada sujeto almacenado en la variable global
for sbj in sorted(global_eeg_promedios.keys()):

    if sbj == 29 or sbj == 34:
        continue
    else:
        data = global_eeg_promedios[sbj]
        left = data['left']    # forma: (canales, muestras, bandas)
        right = data['right']  # forma: (canales, muestras, bandas)
        tv = data['time']

        n_samples = left.shape[1]
        fs = new_fs  # frecuencia de muestreo
        # Generar vector de frecuencias a partir de la FFT
        freqs = np.fft.fftfreq(n_samples, d=1/fs)
        pos_idx = freqs >= 0  # consideramos solo las frecuencias positivas
        pos_freqs = freqs[pos_idx]

        n_bands = left.shape[2]  # número de bandas (por ej. 5)

        # Arrays para almacenar la magnitud en dB para cada banda
        fft_left_db = np.zeros((n_bands, np.sum(pos_idx)))
        fft_right_db = np.zeros((n_bands, np.sum(pos_idx)))

        # Crear figura con 2 filas (izquierda y derecha) y n_bands columnas (una por banda)
        fig, axes = plt.subplots(2, n_bands, figsize=(4*n_bands, 8), sharey=True)

        for b in range(n_bands):
            # Promediar la señal a través de los canales para cada banda
            left_avg = np.mean(left[:, :, b], axis=0)   # vector de longitud = n_samples
            right_avg = np.mean(right[:, :, b], axis=0)

            # Calcular la FFT y obtener la magnitud
            left_fft = np.fft.fft(left_avg)
            right_fft = np.fft.fft(right_avg)
            # Magnitud de la FFT (solo para frecuencias positivas)
            left_mag = np.abs(left_fft)[pos_idx]
            right_mag = np.abs(right_fft)[pos_idx]
            # Convertir a dB (se suma un pequeño epsilon para evitar log(0))
            eps = 1e-12
            left_db = 20 * np.log10(left_mag + eps)
            right_db = 20 * np.log10(right_mag + eps)

            # Almacenar la transformación en los arrays
            fft_left_db[b, :] = left_db
            fft_right_db[b, :] = right_db

            # Graficar en el subplot correspondiente
            axes[0, b].plot(pos_freqs, left_db, color='blue')
            axes[0, b].set_title(f'Mano Izquierda\n{f_bank[b,0]}-{f_bank[b,1]} Hz')
            axes[0, b].set_xlabel('Frecuencia (Hz)')
            axes[0, b].set_ylabel('Amplitud (dB)')

            axes[1, b].plot(pos_freqs, right_db, color='red')
            axes[1, b].set_title(f'Mano Derecha\n{f_bank[b,0]}-{f_bank[b,1]} Hz')
            axes[1, b].set_xlabel('Frecuencia (Hz)')
            axes[1, b].set_ylabel('Amplitud (dB)')

        plt.suptitle(f'Espectro de EEG - Sujeto {sbj} (Promediado sobre canales)')
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.show()

        # Guardar la información de la transformación en la variable global
        # Se agrega un nuevo campo 'fft' que contiene los espectros en dB y el vector de frecuencias
        global_eeg_promedios[sbj]['fft'] = {
            'left': fft_left_db,
            'right': fft_right_db,
            'freqs': pos_freqs
        }


## Ejercicio 5

Discuta las gráficas

Ejercicio 5
Discuta las gráficas

Solucion
Las graficas a analizar corresponden a las 5 ondas principales que estamos utilizando en nuestro estudio, es decir las ondas Gamma, Beta, Alpha,Theta, Delta asi que vamos a analizar cada una por separado

Empezaremos por el ritmo Delta las cuales estan relacionadas con el sueño profundo y la relajacion elevada, estas ondas se encuentran en un rango de frecuencias de 0.5 - 4.0 Hz, en este rango de frecuencias podemos observar los mayores picos de la señal de todas las ondas, sin embargo su oscilacion es pobre y decaen rapidamente, lo que nos puede indicar dos cosas: existe la posibilidad de que sea un pico generado por un ruido leve como un parpadeo o puede que representen la transicion entre un estado de relajacion a un estado de concentracion.

Despues tenemos las ondas Theta las cuales estan relacionadas con procesos de memoria, estados meditativos y estados de relajacion, esta banda de frecuencias presenta un comportamiento similar a las ondas Delta, lo que aumenta la probabilidad de que sea una transicion entre un estado de relajacion a un estado de concentracion el que estamos viendo

Por ultimo tenemos a los ritmos Beta y Gamma, estos son los rangos de frecuencia en los que mas actividad vemos, estas señales estan relacionadas a tareas cognitivas avanzadas, ya que se ha propuesto que la sincronización theta-gamma juega un rol en el procesamiento de información en la corteza y el hipocampo.

Por lo anterior podemos concluir que estas señales estan representando el paso de un estado de relajacion a un estado imaginativo

## Visualización de espectrogramas

Consultar Short Time Fourier Transform



In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
from scipy.signal import stft #
nperseg = 0.5*new_fs#longitud ventas en muestras
vfs,t,Xstft = stft(X,fs=new_fs,nperseg=nperseg,axis=2)
Xstft = 20*np.log10(abs(Xstft))

#graficar stft para un trial y un canal
trail = 0
chi = channels.index('C4')

fig, ax = plt.subplots(2, 1,figsize=(10,6))

ax[1].plot(tv,X[trail,chi,:])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstft[trail,chi])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Esepctrograma EEG Original -- Ch = {channels[chi]}')
print(Xstft.shape)

In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
b = 2
vfs,t,Xstftb = stft(Xrc,fs=new_fs,nperseg=nperseg,axis=2)
Xstftb = 20*np.log10(abs(Xstftb))

print(Xstftb.shape)


fig, ax = plt.subplots(2, 1,figsize=(10,6))
ax[1].plot(tv,Xrc[trail,chi,:,b])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstftb[trail,chi,:,b,:])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Esepctrograma EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]} -- Ch = {channels[chi]}')


# Ejercicio 6

Presente las gráficas de stft para distintos canales en los 5 ritmos cerebrales y discuta.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import stft

# Seleccionar un único sujeto (excluyendo 29 y 34)
all_subjects = sorted(global_eeg_promedios1.keys())
selected_subject = [sbj for sbj in all_subjects if sbj not in [29, 34]][0]
print("Sujeto seleccionado:", selected_subject)

data = global_eeg_promedios1[selected_subject]
# Usamos la condición "mano izquierda"
# left_filtered tiene forma: (n_canales, n_muestras, n_bands) – se asume n_bands = 5
left_data = data['left']
tv = data['time']
fs = new_fs

# Definir los 5 canales deseados para mostrar
desired_channels = ["C3", "Cz", "C4", "FC3", "FC4"]
# Obtener los nombres de los canales filtrados (usando la lista global 'channels' y 'canales_de_interes')
filtered_channels = [channels[i] for i in canales_de_interes]

# Obtener los índices de los canales deseados dentro de los canales filtrados
desired_indices = []
for ch in desired_channels:
    if ch in filtered_channels:
        desired_indices.append(filtered_channels.index(ch))
    else:
        print(f"Advertencia: El canal {ch} no se encuentra entre los canales filtrados.")

n_channels_plot = len(desired_indices)
n_bands = left_data.shape[2]  # Se espera 5 bandas

# Parámetro para STFT: ventana de 0.5 segundos en muestras
nperseg = int(0.5 * new_fs)

# Vamos a crear una figura con 10 filas (2 por cada banda) y n_channels_plot columnas.
# Fila 2*i     : STFT para la banda i
# Fila 2*i + 1 : Señal en dominio del tiempo para la banda i
fig, axes = plt.subplots(2 * n_bands, n_channels_plot, figsize=(4 * n_channels_plot, 2 * 2 * n_bands), sharex=True)

# Asegurarse de que axes es 2D (en caso de n_channels_plot == 1)
if n_channels_plot == 1:
    axes = np.array([[axes[i]] for i in range(2*n_bands)])

# Iterar sobre cada banda (índices 0 a 4)
for band in range(n_bands):
    # Para cada canal deseado
    for idx, col in enumerate(desired_indices):
        # Extraer la señal del canal 'col' para la banda actual
        signal_band = left_data[col, :, band]  # vector de longitud n_samples

        # Calcular la STFT de la señal
        f_vec, t_vec, Zxx = stft(signal_band, fs=fs, nperseg=nperseg)
        eps = 1e-12
        Zxx_db = 20 * np.log10(np.abs(Zxx) + eps)

        # Calcular bordes para pcolormesh (necesarios para shading='flat')
        if len(t_vec) > 1:
            dt = t_vec[1] - t_vec[0]
        else:
            dt = 0.1
        t_edges = np.concatenate((t_vec - dt/2, [t_vec[-1] + dt/2]))

        if len(f_vec) > 1:
            df = f_vec[1] - f_vec[0]
        else:
            df = 1.0
        f_edges = np.concatenate((f_vec - df/2, [f_vec[-1] + df/2]))

        # Ubicar la gráfica de la STFT en la fila 2*band, columna idx
        ax_stft = axes[2*band, idx]
        im = ax_stft.pcolormesh(t_edges, f_edges, Zxx_db, shading='flat', cmap='viridis')
        fig.colorbar(im, ax=ax_stft, orientation='horizontal', pad=0.2)
        ax_stft.set_title(f"{filtered_channels[col]}\n{f_bank[band,0]}-{f_bank[band,1]} Hz")
        ax_stft.set_xlabel("Tiempo [s]")
        ax_stft.set_ylabel("Frecuencia [Hz]")

        # Graficar la señal en el dominio del tiempo en la fila 2*band+1
        ax_time = axes[2*band + 1, idx]
        ax_time.plot(tv, signal_band)
        ax_time.set_title(f"Señal {filtered_channels[col]}")
        ax_time.set_xlabel("Tiempo [s]")
        ax_time.set_ylabel("Amplitud [$\mu$V]")

fig.suptitle(f"STFT en Bloques para las 5 Bandas - Sujeto {selected_subject} (Mano Izquierda)", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


## Visualización de señales EEG sobre montaje 10-20

In [ ]:
import mne

# Cargar el montaje estándar
easycap_montage = mne.channels.make_standard_montage("standard_1020")


# Crear un montaje personalizado con los electrodos seleccionados
custom_pos = {ch: easycap_montage.get_positions()["ch_pos"][ch] for ch in channels}
custom_montage = mne.channels.make_dig_montage(ch_pos=custom_pos, coord_frame="head")

# Mostrar el montaje personalizado
custom_montage.plot(show_names=True)
fig = custom_montage.plot(kind="3d", show_names=True, show=False)
fig.gca().view_init(azim=70, elev=15)  # Ajustar la vista 3D

In [ ]:
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.visualizations.git

# Topomaps

In [ ]:

from gcpds.visualizations.topoplots import topoplot
import mne

# Cargar el montaje estándar
easycap_montage = mne.channels.make_standard_montage("standard_1020")


# Crear un montaje personalizado con los electrodos seleccionados
custom_pos = {ch: easycap_montage.get_positions()["ch_pos"][ch] for ch in channels}
custom_montage = mne.channels.make_dig_montage(ch_pos=custom_pos, coord_frame="head")

# Mostrar el montaje personalizado
custom_montage.plot(show_names=True)
fig = custom_montage.plot(kind="3d", show_names=True, show=False)
fig.gca().view_init(azim=70, elev=15)  # Ajustar la vista 3D

trial = 150
vec_topo_o = abs(X[trial,:]).mean(axis=-1)
vec_topo_b = abs(Xrc[trial,:,:,:]).mean(axis=1)


fig,ax = plt.subplots(1,6,figsize=(20,10))
topoplot(vec_topo_o, channels, contours=3, cmap='Reds', names=channels, sensors=False,ax=ax[0],show=False,vlim=(min(vec_topo_o), max(vec_topo_o)))

for b in range(f_bank.shape[0]):
    vec_ = vec_topo_b[:,b]
    topoplot(vec_, channels, contours=3, cmap='Reds', names=channels, sensors=False,ax=ax[b+1],show=False,vlim=(min(vec_), max(vec_)))
    ax[b+1].set_title(ritmo[b])

ax[0].set_title(f'EEG-suj={sbj}-trial={trial}')

plt.show()

## Ejercicio 7

Discuta

La banda alpha está asociada con la relajación y la tranquilidad mental. Si en nuestras gráficas vemos mucha actividad en la parte trasera de la cabeza, eso indica que el cerebro está en un estado de calma, sin procesar demasiada información externa. Sin embargo, si el experimento consiste en imaginar un movimiento, lo normal es que esta actividad disminuya en la parte superior del cerebro, señal de que la persona está concentrada en la tarea. En otras palabras, cuando alguien se enfoca en mover algo con la mente, su cerebro sale de ese estado de reposo y comienza a activarse.

Por su parte, la banda beta está relacionada con la concentración y el pensamiento activo. Si en la gráfica aparece mucha actividad en la parte frontal y superior de la cabeza, significa que el cerebro está trabajando intensamente, tal vez planeando un movimiento o procesando información. Si esta actividad beta es más fuerte en los lados superiores, es una señal de que la persona está pensando en mover su cuerpo, aunque no lo haga físicamente. Además, si vemos que la banda alpha baja en esas zonas mientras la beta sube, podemos decir con más certeza que la persona realmente está enfocada en imaginar una acción.

## Common Spatial Patterns

In [ ]:
import mne
from mne.decoding import CSP

# Instancia del objeto CSP
n_components = 2
csp = CSP(n_components=n_components, log= True, transform_into='average_power')
# Ajuste y transformación de los datos
csp_data = csp.fit_transform(X.astype(np.float64), y)

In [ ]:
print("CSP Transformado Shape:", csp_data.shape)
plt.scatter(csp_data[:,0],csp_data[:,1],c=y)
plt.show()

In [ ]:
#EEG original
fig,ax = plt.subplots(1,n_components,figsize=(5,5))
for cc in range(n_components):
    vec_ = np.abs(csp.filters_[cc])
    topoplot(vec_, channels, contours=3, cmap='Reds', names=channels, sensors=False,ax=ax[cc],show=False,vlim=(min(vec_), max(vec_)))
    ax[cc].set_title(f'CSP {cc+1}')


In [ ]:
#lectura de datos
sbj = 2
X, y = load_GIGA(sbj=sbj, **load_args)

f_bank = np.array([[0.5,4.],[4., 8.],[8.,13.],[13.,32.],[32.,100.]])
vwt = np.array([[0.25, 1.75],[1.5,3],[2.75,4.25],[4,5.5],[5.25,6.75]]) #2.5 - 5 MI 0 - 7 trial completo
tf_repr = TimeFrequencyRpr(sfreq = new_fs, f_bank = f_bank,vwt=vwt)
X_ = np.squeeze(tf_repr.transform(X))
X_.shape

In [ ]:
# csp por ventanas y ritmos
# Definir las dimensiones del arreglo
ritmos_ = f_bank.shape[0]
ventanas_ = vwt.shape[0]
n_comp = 2
# Inicializar el arreglo vacío con listas anidadas
csp_M = [[None for _ in range(ventanas_)] for _ in range(ritmos_)]
csp_filters_ = np.zeros((ritmos_,ventanas_,X_.shape[1],X_.shape[1])) #ritmos ventanas Ch
Xcsp_ = np.zeros((X_.shape[0],n_comp,ritmos_,ventanas_))

for i in range(ritmos_):
    for j in range(ventanas_):
        print(f'CSP ritmo {f_bank[i]} -- ventana {vwt[j]}...')
        csp_M[i][j] =  CSP(n_components=n_comp, log= True, transform_into='average_power')
        Xcsp_[:,:,i,j] = csp.fit_transform(X_[:,:,:,j,i].astype(np.float64), y)
        csp_filters_[i,j,:] = np.abs(csp.filters_)

In [ ]:
# graficar topomaps
fig, ax = plt.subplots(ritmos_,ventanas_,figsize=(12,12))

for i in range(ritmos_):
    for j in range(ventanas_):
        vec_ = csp_filters_[i,j,0]
        vec_ = vec_/max(vec_)
        topoplot(vec_, channels, contours=3, cmap='Reds', names=None, sensors=False,ax=ax[i,j],show=False,vlim=(min(vec_), max(vec_)))
    ax[i,0].set_ylabel(ritmo[i],fontsize=20)
for j in range(ventanas_):
     ax[0,j].set_title(f'{vwt[j,0]}--{vwt[j,1]} [s]',fontsize=15)

plt.subplots_adjust(hspace=-0.025,wspace=-0.025)
plt.show()

In [ ]:
# Graficar topomaps
fig, ax = plt.subplots(ritmos_, ventanas_, figsize=(12, 12))

for i in range(ritmos_):
    for j in range(ventanas_):
        vec_ = csp_filters_[i, j, 0]
        vec_ = vec_ / max(vec_)  # Normalizar el vector
        topoplot(vec_, channels, contours=3, cmap='Reds', names=None, sensors=False,
                 ax=ax[i, j], show=False, vlim=(min(vec_), max(vec_)))
    ax[i, 0].set_ylabel(ritmo[i], fontsize=20)

for j in range(ventanas_):
    ax[0, j].set_title(f'{vwt[j, 0]}--{vwt[j, 1]} [s]', fontsize=15)

plt.subplots_adjust(hspace=-0.025, wspace=-0.025)
plt.show()


In [ ]:
#scatters
fig, ax = plt.subplots(ritmos_,ventanas_,figsize=(12,12))

for i in range(ritmos_):
    for j in range(ventanas_):
        ax[i,j].scatter(Xcsp_[:,0,i,j],Xcsp_[:,1,i,j],c=y)
        ax[i,j].set_xticks([])
        ax[i,j].set_yticks([])
    ax[i,0].set_ylabel(ritmo[i],fontsize=20)
for j in range(ventanas_):
     ax[0,j].set_title(f'{vwt[j,0]}--{vwt[j,1]} [s]',fontsize=15)

plt.subplots_adjust(hspace=0.1,wspace=0.1)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gcpds.visualizations.topoplots import topoplot

# Índices de banda en el 3er eje
BAND_ALPHA = 2
BAND_BETA  = 3

# Ventanas de tiempo (cada columna)
time_windows = [
    (0.25, 1.75),
    (1.5,  3.0),
    (2.75, 4.25),
    (4.0,  5.5),
    (5.25, 6.75),
]

# Nombre de filas (row_labels):
row_labels = ["alpha-left", "beta-left", "alpha-right", "beta-right"]

# Recorremos los sujetos de la variable global
for sbj in sorted(global_eeg_promedios.keys()):
    data = global_eeg_promedios[sbj]

    # Extraer las señales para mano izquierda y derecha
    left_data  = data['left']   # (n_canales, n_muestras, n_bands)
    right_data = data['right']  # (n_canales, n_muestras, n_bands)
    tv         = data['time']   # (n_muestras,)

    # Crear la figura con 4 filas (alpha-left, beta-left, alpha-right, beta-right)
    # y 5 columnas (ventanas)
    fig, axes = plt.subplots(4, 5, figsize=(14, 10), sharex=True, sharey=True)

    # Bucle sobre las 5 columnas
    for col, (start_t, end_t) in enumerate(time_windows):
        # ÍNDICES de muestras en la ventana [start_t, end_t)
        idx_time = np.where((tv >= start_t) & (tv < end_t))[0]

        # Si no hay muestras en esa ventana, se deja la columna en blanco
        if len(idx_time) == 0:
            for row in range(4):
                axes[row, col].set_title(f"{start_t}-{end_t} s\n(No data)")
                axes[row, col].axis("off")
            continue

        # Calcular la magnitud promedio en la ventana => (n_canales, n_bands)
        left_vec  = np.abs(left_data[:, idx_time, :]).mean(axis=1)
        right_vec = np.abs(right_data[:, idx_time, :]).mean(axis=1)

        # Extraer alpha y beta para cada mano
        alpha_left  = left_vec[:, BAND_ALPHA]
        beta_left   = left_vec[:, BAND_BETA]
        alpha_right = right_vec[:, BAND_ALPHA]
        beta_right  = right_vec[:, BAND_BETA]

        # -------------- Fila 0: alpha-left --------------
        vmin, vmax = alpha_left.min(), alpha_left.max()
        topoplot(alpha_left, channels,
                 ax=axes[0, col],
                 show=False,
                 sensors=False,
                 cmap='viridis',  # alpha en viridis
                 vlim=(vmin, vmax))
        # Etiqueta
        axes[0, col].set_title(f"{start_t}-{end_t} s")

        # -------------- Fila 1: beta-left --------------
        vmin, vmax = beta_left.min(), beta_left.max()
        topoplot(beta_left, channels,
                 ax=axes[1, col],
                 show=False,
                 sensors=False,
                 cmap='Reds',  # beta en Reds
                 vlim=(vmin, vmax))

        # -------------- Fila 2: alpha-right --------------
        vmin, vmax = alpha_right.min(), alpha_right.max()
        topoplot(alpha_right, channels,
                 ax=axes[2, col],
                 show=False,
                 sensors=False,
                 cmap='viridis',
                 vlim=(vmin, vmax))

        # -------------- Fila 3: beta-right --------------
        vmin, vmax = beta_right.min(), beta_right.max()
        topoplot(beta_right, channels,
                 ax=axes[3, col],
                 show=False,
                 sensors=False,
                 cmap='Reds',
                 vlim=(vmin, vmax))

    # Etiquetas de filas
    for row in range(4):
        axes[row, 0].set_ylabel(row_labels[row], fontsize=12)

    fig.suptitle(f"Sujeto {sbj} - Alpha/Beta en 5 ventanas\n(Arriba=Izquierda, Abajo=Derecha)", fontsize=16)
    plt.tight_layout()
    plt.show()


In [ ]:
# calculamos las matrices de coectividades (correlación)

N = X.shape[0]
connectivities_v = np.array([np.corrcoef(X[i]) for i in range(N)])

print(connectivities_v.shape)

In [ ]:
from gcpds.visualizations.connectivities import CircosConnectivity

N = len(channels)  # channels

areas = {
    'Frontal': ['Fpz', 'AFz', 'Fz', 'FCz'],
    'Frontal Right': ['Fp2','AF4','AF8','F2','F4','F6','F8',],
    'Central Right': ['FC2','FC4','FC6','FT8','C2','C4','C6','T8','CP2','CP4','CP6','TP8',],
    'Posterior Right': ['P2','P4','P6','P8','P10','PO4','PO8','O2',],
    'Central': ['Cz'],
    'Posterior': ['CPz','Pz', 'Cz','POz','Oz','Iz',],
    'Posterior Left': ['P1','P3','P5','P7','P9','PO3','PO7','O1',],
    'Central Left': ['FC1','FC3','FC5','FT7','C1','C3','C5','T7','CP1','CP3','CP5','TP7',],
    'Frontal Left': ['Fp1','AF3','AF7','F1','F3','F5','F7',],
}


conn = CircosConnectivity(
    connectivities_v[0,:,:], channels, areas=areas, threshold=0.7,

    # cmaps and themes
    areas_cmap='Set3',
    arcs_cmap='Wistia',
    hemisphere_color='lightgray',
    channel_color='#f8f9fa',
    min_alpha=0,


    # Texts
    width={'hemispheres':35, 'areas':100, 'channels':60},
    text={'hemispheres':40, 'areas':20,  'channels':40},
    separation={'hemispheres':10, 'areas':-30, 'channels':5},
    labelposition={'hemispheres':60, 'areas':0, 'channels':-10},
    size=10,
    labelsize=15,


    # Shapes
    show_emisphere=True,
    arcs_separation=30,
    connection_width=0.1,
    small_separation=5,
    big_separation=10,
    offset=0,
)

conn.figure;


## Topoplots

In [ ]:
# promediemos por canales

connecitivities_mean = np.mean(connectivities_v[0],axis=1)

## Graficamos las bandas de frecuencia usando fourier para el calculo